<a href="https://colab.research.google.com/github/Yirgaalem/CP322-Machine-Learning/blob/main/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# CP 322– Machine Learning – Fall 2022
## Assignment 2
### Due: Nov 4th, 2022, 11:59 pm

Ojective of this assignment:

Application of ML in analyzing text documents:

In this asssignment, we take advantage of scikit-learn in working with text documents. This will also be an excercise to figure out how to write a code with a new machine learning package; this is a necessary skill in applied machine learning, since the packages evolve quickly (and there are many of them) so being able to figure out how to work with a tool within a reasonble time frame is important. If you need further details you can check out to this <a href="https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html" > scikit example </a>, or other scikit documentations.

# Procedure:

You need to make a copy of this notebook and download the dataset file. Write the answar in the provided space.

For submission, you need to upload this notebook in Assignment2 dropbox section.
- There are three tasks for you.
- Submit your code and justify your results in the same Jupiter notebook format. 
   (keep the overal format of the notebook unchanged)

Rename the submitted file as Assignment2_student_id.

Make a copy of this colab so that you can modify it for yourself. If google colab is slow, you can also download the notebook and use Jupyter notebook on your computer. Using the online notebook has the benefit that all the required packages are already installed.


# Packages

First of all, let's import the packages we need for this assignment.


In [ ]:
# loading need libraries
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import nltk
import re
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split

# Dataset characteristics

The dataset for this work is available for download from the [Cornell Natural Language Processing Group](https://www.cs.cornell.edu/people/pabo/movie-review-data/). There are a total of 2000 documents in the collection. The favourable ratings are found in half of the documents, while the negative reviews are found in the other half. This site has further information on the dataset. If you are unable to download the dataset for the above link, download the 
txt_sentoken.zip file of the dataset in the assignment 2 folder, unzip it and change the path in the load_files function below.

In [ ]:
#!wget http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz
#!tar xzf review_polarity.tar.gz
movie_data = load_files("/content/txt_sentoken") # Change the path according to your PC path.
reviews, labels = movie_data.data, movie_data.target


In [ ]:
print("Dataset properties :")
print("\t Number of reviews: %d" % len(reviews))
print("\t Number of Classes: %d" % (max(labels)+1))
reviews_train, reviews_test, labels_train, labels_test = train_test_split(reviews, labels, test_size=0.2, random_state=0)
print("\t Number of reviews for training set: %d" % len(reviews_train))
print("\t Number of reviews for test set: %d" % len(reviews_test))

Dataset properties :
	 Number of reviews: 2000
	 Number of Classes: 3
	 Number of reviews for training set: 1600
	 Number of reviews for test set: 400


# Text Preprocessing 
The next step is to preprocess the text after the dataset has been imported.
Numbers, special characters, and undesired spaces are all possible in text.
We may or may not need to delete certain special letters and numbers from text, depending on the situation. We shall, however, delete any special characters, digits, and unnecessary spaces from our text for the purpose of clarity.
To preprocess the data, run the script below: 

In [ ]:
from nltk.stem import WordNetLemmatizer

def text_preprocessing(input):
  # Remove all the special characters
    document = re.sub(r'\W', ' ', str(input))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    return document
stemmer = WordNetLemmatizer()
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...


True

**Importance of preprocessing:**
The following is an example of text preparation in action:

In [ ]:
print('before preprocessing: \n',reviews_train[0])

before preprocessing: 
 b'it was a rainy friday afternoon in columbus when i persuaded a friend to see a matinee performance of mst3k : tm . \nhe had never seen any episodes of the show , and i have watched a scant few , due to its unsocial airtime on comedy central and the uneven nature of many of the episodes . \nfor those of you not familiar with the premise , dr clayton forrester ( beaulieu ) wishes to take over the world by finding the worst film ever made and unleashing it upon an unsuspecting public . \nto achieve this , he ( in the words of the tv series\' theme , which is missing in the movie ) " bumped [mike nelson ( nelson ) ] on the noggin and then shot him into space " , and is monitoring nelson\'s reactions to the movies he is forced to endure . \nrather than succumb to the sheer awfulness of many of the movies , nelson spends his time making wisecracks with the help of his two robot companions , tom servo and crow t . robot . \nthe format of the show consists of nelson ,

In [ ]:
import nltk
nltk.download('omw-1.4')
print('after preprocessing: \n',text_preprocessing(reviews_train[0]))

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


after preprocessing: 
 it wa rainy friday afternoon in columbus when persuaded friend to see matinee performance of mst3k tm nhe had never seen any episode of the show and have watched scant few due to it unsocial airtime on comedy central and the uneven nature of many of the episode nfor those of you not familiar with the premise dr clayton forrester beaulieu wish to take over the world by finding the worst film ever made and unleashing it upon an unsuspecting public nto achieve this he in the word of the tv series theme which is missing in the movie bumped mike nelson nelson on the noggin and then shot him into space and is monitoring nelson reaction to the movie he is forced to endure nrather than succumb to the sheer awfulness of many of the movie nelson spends his time making wisecrack with the help of his two robot companion tom servo and crow robot nthe format of the show consists of nelson servo and crow making their comment while silhouetted against the movie being watched and

# Apply pre-processing step on the entire dataset


In [ ]:
pre_processed_reviews_train = [text_preprocessing(rev) for rev in reviews_train]
pre_processed_reviews_test = [text_preprocessing(rev) for rev in reviews_test]

# Feature extraction
since our data is text, to run classification models on the dataset we will turn them into vectors with numerical features. Therefore, in this section, we extract features using the **Bag of Words** method. To this regard, 


*   Assign an integer ID to each word in the dataset (like a dictionary).
*   For each data point ( document i), count the number of occurances of word w and put it in X[i,j] where i is the i'th document and j is the index of the word w in the dictionary.
Thus, if we have e.g., 10000 data points and 100000 words in the dictionary, then X will be a 10,000 by 100,000 matrix, which is huge! The good news is that most elements of the matrix X are zero (not all the words are used in every document). Therefore, it is possible to (somehow) just store non-zero elements and save up a lot of memory. Fortunately, the library that we use supports using "sparse" data representations, meaning that it does not actually store all the zero-values.
# Tokenizing with scikit-learn
In the following part we extract whole words that have been used in the dataset and compute their occurance count in each document. This shows number of documents are **1600** and number of features (unique words in the whole set of documents) is **36551**.


In [ ]:
count_vect = CountVectorizer()
X_counts_train = count_vect.fit_transform(np.array(pre_processed_reviews_train))
print(X_counts_train.shape)

(1600, 36551)


Up to here, we turned each document into an occurrence feature map (i.e., bag of words representation). But there is an issue with this solution: longer documents tend to have larger occurrence values. This is not ideal; for example, if we just repeat the same text twice, we don't expect the category of that document to change, but the occurance values will drastically change.


Solution: we better normalize each document by dividing the occurrence values of each word by the total number of words in the document (*tf* normalization, where tf stands for term-frequency).

Another issue is that we have some words that are so common that do not give much information (think of words like "is", "the", etc.). In order to reduce the effect of those words, one can use the *tf-idf* method, where on top of normalizing based on the length of the documents (*tf*), we also downscale weights for words that are presented in many documents (*idf* stands for inverse document frequency)

If you are interested to know more about tf-idf, feel free to check out the wikipedia page. 

For this assignment, we will use *tf* and also *tf-idf* noramalization.

The below application of ***TfidfTransformer*** is showed when idf is turned off. Evidently, we don't observe any changes in our feature dimension after performing **tf-idf** step.

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer(use_idf=False)
X_tfidf_train = tfidfconverter.fit_transform(X_counts_train).toarray()


# Document classification
K-nearest neighbours (KNNs) are one of the most common classifiers in practices. Here we train an KNN classifier on the transformed features.

In [ ]:

classifier1=KNeighborsClassifier(n_neighbors=2)
classifier1.fit(X_tfidf_train, labels_train)
X_counts_test = count_vect.transform(np.array(pre_processed_reviews_test))
X_tfidf_test = tfidfconverter.transform(X_counts_test).toarray()
predicted = classifier1.predict(X_tfidf_test)
print('Accuracy:{}'.format(np.mean(predicted == labels_test)))

Accuracy:0.6425


# Pipeline
We can do three tasks (vectorizer,transformer,classifier) using pipeline procedure.
We can create a "pipeline" for performing a sequence of steps, namely first extracting the words and creating vectors, then using tf or tf-idf, and then training the classifier. This helps to make our code cleaner (and allows for more code optimization, etc.) We utilize a pipeline to do vectorizer -> transformer -> classifier

In [ ]:
text_classifier1 = Pipeline([
      ('vect', CountVectorizer()),
      ('tfidf', TfidfTransformer(use_idf=False)),
      ('classifier1', KNeighborsClassifier(n_neighbors=2)),
  ])
text_classifier1.fit(pre_processed_reviews_train, labels_train)
docs_test = pre_processed_reviews_test
predicted = text_classifier1.predict(docs_test)
print('Accuracy:{}'.format(np.mean(predicted ==  labels_test)))

Accuracy:0.6425


# <font color="red">Task 1 (15 points) </font>

## Logistic Regression Classifier:

In this part, we will test Logistic Regression(LR) with three distinct penalties: 'none', 'l1' and 'l2'. Report test accuracies and justify the result. The solver can be set to 'saga' to support both types of penalties. The details about LR will be found on 
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html


#sklearn.linear_model.LogisticRegression

Please use the following settings:

count_vect = CountVectorizer(max_features=100)




In [ ]:
# write your code here
# In this task we see that the "none" penalty has the highest accuracy
# l1 penalizes the sum of absolute values of the weights whereas l2 peanlizes the sum of squares of the weights 
# Tfidf goal is to is to scale down the impact of the tokens

# L1 and L2 are regularizations that can be used to address an overfitting issue

# The reason in Task1, the penalty 'none' has the highest accuracy is because a peanlty tries to reduce overfitting. So no peantly being added in this task, is ideal as we are not looking
# weights of data or anythig like that. Here an overfitting would be good.
# Overfitting gives high accuracy when all we have is the training data, however, doesn't do well with a test set or some other data.

penalties = ["none", "l1", "l2"]

for pen in penalties:
  text_classifier1 = Pipeline([
      ('vect', CountVectorizer(max_features = 100)),
      ('tfidf', TfidfTransformer(use_idf=False)),
      ('classifier1', LogisticRegression(penalty = pen, solver = 'saga', max_iter=1000)),
  ])
  text_classifier1.fit(pre_processed_reviews_train, labels_train)
  docs_test = pre_processed_reviews_test
  predicted = text_classifier1.predict(docs_test)
  print('Accuracy penalty "{}" :{}'.format(pen,np.mean(predicted ==  labels_test))) 


Accuracy penalty "none" :0.7175
Accuracy penalty "l1" :0.7
Accuracy penalty "l2" :0.69


# <font color="red">Task 2 (5 points) </font>
 
## idf importance:
  
How would the results of Task 1 change if we turn on TfidfTransformer(use_idf=True)?
    
Report the results on the test set and justify them.
Please use the following settings:
        
count_vect = CountVectorizer(max_features=1500)

Tfidfconverter = TfidfTransformer(use_idf=True)

In [ ]:
# write your code here
#In this task we see that either penalties l1 and l2 have the same accuracy or l1 is larger then l2 (in both cases none has the lowest)
#Tf-idf's goal is scale down the ipmact of tokens that occur very frequently. It takes a string and quantifies the importance of the string
#l1 shrinks coeefficients to zero whereas l2 shrinks coefficients evenly. 

#The reason which the 'none' penalty has the lowest accuracy is because when TfidTransformer is set to true, as we are now considering the weight of the data, the penalty does nothing to 
#change any of the data points weight. As explained earlier, l1 shrinks the coefficients to zero and l2 shrinks coefficients evenly. The reason l1 has a higher accuracy then l2 is because 
#l1 creates a larger difference between weights of different strings. Shrinking it to 0 compared to shrinking them evenly like l2 allows for the algorithm to see who is more accurate. However,
#the difference of accuracy is not a very large, amount, sometimes so little the output states they are equal
penalties = ["none", "l1", "l2"]


for pen in penalties:
  text_classifier1 = Pipeline([
      ('vect', CountVectorizer(max_features = 1500)),
      ('tfidf', TfidfTransformer(use_idf=True)),
      ('classifier1', LogisticRegression(penalty = pen, solver = 'saga', max_iter=1600)),
  ])
  text_classifier1.fit(pre_processed_reviews_train, labels_train)
  docs_test = pre_processed_reviews_test
  predicted = text_classifier1.predict(docs_test)
  print('Accuracy penalty "{}" :{}'.format(pen,np.mean(predicted ==  labels_test))) 


Accuracy penalty "none" :0.79
Accuracy penalty "l1" :0.805
Accuracy penalty "l2" :0.8


 # <font color="red">Task 3 (10 points) </font>

## Importance of maximum feature size:

Examine the vocabulary size, max feature choices for CountVectorizer: 100, 500, 1000, 1500, and 2000. 
Which size of vocabulary yields the greatest results on the test samples? Justify your answer. 
Please use the following settings:

tfidfconverter = TfidfTransformer(use_idf=True)

In [28]:
# write your code here
#The greateer the countVectorSize, the higher the accuracy, meaning, that 2000 countVectorSize yields teh greatest results on the test samples
penalties = ["none", "l1", "l2"]
countVectorSize = [100, 500, 1000, 1500, 2000]

for count in countVectorSize:
  print('\nCount Vector Size = "{}"'.format(count))
  for pen in penalties:
    max = 2000
    
    text_classifier1 = Pipeline([
      ('vect', CountVectorizer(max_features = count)),
      ('tfidf', TfidfTransformer(use_idf=True)),
      ('classifier1', LogisticRegression(penalty = pen, solver = 'saga', max_iter=max)),
    ])
    text_classifier1.fit(pre_processed_reviews_train, labels_train)
    docs_test = pre_processed_reviews_test
    predicted = text_classifier1.predict(docs_test)
    print('Accuracy penalty "{}": {}'.format(pen,np.mean(predicted ==  labels_test))) 
  




Count Vector Size = "100"
Accuracy penalty "none": 0.715
Accuracy penalty "l1": 0.71
Accuracy penalty "l2": 0.7075

Count Vector Size = "500"


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


Accuracy penalty "none": 0.74
Accuracy penalty "l1": 0.795
Accuracy penalty "l2": 0.7775

Count Vector Size = "1000"
Accuracy penalty "none": 0.775
Accuracy penalty "l1": 0.8025
Accuracy penalty "l2": 0.7975

Count Vector Size = "1500"
Accuracy penalty "none": 0.7925
Accuracy penalty "l1": 0.8025
Accuracy penalty "l2": 0.8

Count Vector Size = "2000"
Accuracy penalty "none": 0.8325
Accuracy penalty "l1": 0.8025
Accuracy penalty "l2": 0.8025
